#Experimentos de dropout
Com **512, 1024** [N]euronios. **1, 2, 3** [L]ayers. **Sem, All, 1 Last Dense 1 e Last CNN, Só All Dense, Só Last Dense, Só All CNN, Só Last CNN** [D]ropout.

Parametros:
* Batch	64
* epocas	100
* LR	1,00E-05
* Optimizer	Adam

Parametros Early Stopping:
* Monitor	val_auc
* Min_delta	0
* Patience	5
* verbose	1
* mode	auto
* baseline	none
* restore_best_weights	True

# Imports e Installs

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Rescaling
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import TensorBoard

!pip install pydicom
import numpy as np
import pydicom
from PIL import Image
import os

import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 28.6 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Ajustando o DS

## Baixando o DS

In [ ]:
!kaggle datasets download -d pedroamaro/smaller-rsna-ds-train-test

100% 7.25G/7.25G [03:00<00:00, 70.6MB/s]
100% 7.25G/7.25G [03:00<00:00, 43.2MB/s]


In [ ]:
!unzip /content/smaller-rsna-ds-train-test.zip | awk 'BEGIN {ORS=" "} {if(NR%1000==0)print "."}'
!rm /content/smaller-rsna-ds-train-test.zip

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_train/no | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_train/yes | wc -l

Numero de imagens: NO
20047
Numero de imagens: YES
3287


## Salvando como JPG e aplicando full windowing

In [ ]:
!mkdir small_rsna_ds_as_jpg
!mkdir /content/small_rsna_ds_as_jpg/no
!mkdir /content/small_rsna_ds_as_jpg/yes

!mkdir test_as_jpg
!mkdir test_as_jpg/no
!mkdir test_as_jpg/yes

In [ ]:
def segment_circle(windowed):
    original = windowed.copy().astype("uint8")
    mask = np.zeros(original.shape, dtype=np.uint8)
    gray = original
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=5)
    cnts = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        area = cv2.contourArea(c)
        if len(approx) > 4 and area > 10000 and area < 500000:
            ((x, y), r) = cv2.minEnclosingCircle(c)
            cv2.circle(mask, (int(x), int(y)), int(r), (255, 255, 255), -1)
            cv2.circle(original, (int(x), int(y)), int(r), (36, 255, 12), 0)
    x,y,w,h = cv2.boundingRect(mask)
    mask_ROI = mask[y:y+h, x:x+w]
    image_ROI = original[y:y+h, x:x+w]
    result = cv2.bitwise_and(image_ROI, image_ROI, mask=mask_ROI)
    return result, (x, y, w, h)

def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(img, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img


def preprocess_and_segment(path, make_correct=True):    
    dcm = pydicom.dcmread(path)
    if make_correct and (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)

    sample = dcm.pixel_array.astype("float32") * dcm.RescaleSlope + dcm.RescaleIntercept

    brain_window = window_image(sample, 40, 80)
    dural_window = window_image(sample, 80, 200)
    bone_window = window_image(sample, 600, 2800)

    brain_window, (x,y,w,h) = segment_circle(brain_window)
    if brain_window is None:
        return np.zeros((512, 512, 3))
    dural_window = dural_window[y:y+h, x:x+w]
    bone_window = bone_window[y:y+h, x:x+w]
    
    brain_window = (brain_window - (0.)) / 80.
    dural_window = (dural_window - (-20.)) / 200.
    bone_window = (bone_window - (-1200.)) / 2800.
    img_3ch = np.dstack([brain_window, dural_window, bone_window]).astype("float32")

    return img_3ch


def save_W_as_jpg(path, id, dest_folder):

  im_3ch = preprocess_and_segment(path)

  im = window_image(im_3ch, 40, 80)

  rescaled_im = (np.maximum(im, 0)/im.max())*255.0
  final_im = np.uint8(rescaled_im)

  final_im = Image.fromarray(final_im)
  final_im.save(dest_folder+id+'.jpg')

### Para o conjunto Train

In [ ]:
train_path = "/content/small_rsna_ds_train/yes/"
folder = os.listdir("/content/small_rsna_ds_train/yes/")
dest_folder = "/content/small_rsna_ds_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
train_path = "/content/small_rsna_ds_train/no/"
folder = os.listdir("/content/small_rsna_ds_train/no/")
dest_folder = "/content/small_rsna_ds_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_as_jpg/yes/ | wc -l

### Para o conjunto Test

In [ ]:
train_path = "/content/small_rsna_ds_test/yes/"
folder = os.listdir("/content/small_rsna_ds_test/yes/")
dest_folder = "/content/test_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

In [ ]:
train_path = "/content/small_rsna_ds_test/no/"
folder = os.listdir("/content/small_rsna_ds_test/no/")
dest_folder = "/content/test_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

In [ ]:
print('Numero de imagens: NO')
!ls /content/test_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/test_as_jpg/yes/ | wc -l

Numero de imagens: NO
10022
Numero de imagens: YES
1643


# Carregando o dataset

In [ ]:
train_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="training",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
     )

In [ ]:
val_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="validation",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
)

In [ ]:
test_data = image_dataset_from_directory(
    "/content/test_as_jpg",
     labels='inferred',
     label_mode="int",
     batch_size = 64,
     color_mode = "rgb"
     )

In [ ]:
class_names = train_data.class_names

## Normalizando e Pegando o tamanho de entrada

In [ ]:
for image_batch, labels_batch in train_data:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(64, 256, 256, 3)
(64,)


In [ ]:
normalization_layer = Rescaling(1./255)

normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_data))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [ ]:
normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
val_image_batch, val_labels_batch = next(iter(normalized_val_data))
first_image = val_image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [ ]:
normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
test_image_batch, test_labels_batch = next(iter(normalized_test_data))
first_image = test_image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


# Sem class_weights

#### N512-L1-D3

In [ ]:
N = 512
NAME = "N512-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Normalizados/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 70s 263ms/step - loss: 0.4068 - auc: 0.6381 - accuracy: 0.8589 - val_loss: 0.3561 - val_auc: 0.7695 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 64s 258ms/step - loss: 0.3549 - auc: 0.7537 - accuracy: 0.8597 - val_loss: 0.3471 - val_auc: 0.7864 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 58s 237ms/step - loss: 0.3489 - auc: 0.7677 - accuracy: 0.8597 - val_loss: 0.3436 - val_auc: 0.7914 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 64s 260ms/step - loss: 0.3426 - auc: 0.7806 - accuracy: 0.8596 - val_loss: 0.3377 - val_auc: 0.7975 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 64s 260ms/step - loss: 0.3380 - auc: 0.7897 - accuracy: 0.8598 - val_loss: 0.3356 - val_auc: 0.7990 - val_accuracy: 0.8582
Epoch 6/100
244/244 [==============================] - 64s 260ms/step - loss: 0.3351 - auc: 0.7945 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 22s 118ms/step - loss: 0.3200 - auc: 0.8120 - accuracy: 0.8742
[0.3199867606163025, 0.8119522333145142, 0.8742392063140869]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Normalizados/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### N512-L3-D2

In [ ]:
N = 512
NAME = "N512-L3-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Normalizados/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 58s 232ms/step - loss: 0.4258 - auc: 0.5920 - accuracy: 0.8578 - val_loss: 0.3648 - val_auc: 0.7524 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 57s 232ms/step - loss: 0.3623 - auc: 0.7323 - accuracy: 0.8597 - val_loss: 0.3511 - val_auc: 0.7775 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 57s 230ms/step - loss: 0.3525 - auc: 0.7587 - accuracy: 0.8597 - val_loss: 0.3429 - val_auc: 0.7896 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 57s 229ms/step - loss: 0.3457 - auc: 0.7722 - accuracy: 0.8610 - val_loss: 0.3407 - val_auc: 0.7937 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 65s 265ms/step - loss: 0.3398 - auc: 0.7835 - accuracy: 0.8622 - val_loss: 0.3365 - val_auc: 0.7980 - val_accuracy: 0.8592
Epoch 6/100
244/244 [==============================] - 65s 263ms/step - loss: 0.3342 - auc: 0.7932 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Normalizados/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 20s 105ms/step - loss: 0.3242 - auc: 0.8080 - accuracy: 0.8724
[0.3242402672767639, 0.8079959750175476, 0.872438907623291]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Normalizados/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### N1024-L1-D4

In [ ]:
N = 1024
NAME = "N1024-L1-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Normalizados/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 69s 278ms/step - loss: 0.4066 - auc: 0.6357 - accuracy: 0.8596 - val_loss: 0.3601 - val_auc: 0.7541 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 61s 245ms/step - loss: 0.3595 - auc: 0.7403 - accuracy: 0.8597 - val_loss: 0.3509 - val_auc: 0.7762 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 68s 276ms/step - loss: 0.3517 - auc: 0.7629 - accuracy: 0.8597 - val_loss: 0.3464 - val_auc: 0.7876 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 69s 279ms/step - loss: 0.3458 - auc: 0.7762 - accuracy: 0.8597 - val_loss: 0.3413 - val_auc: 0.7925 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 60s 244ms/step - loss: 0.3403 - auc: 0.7862 - accuracy: 0.8597 - val_loss: 0.3378 - val_auc: 0.7966 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 69s 278ms/step - loss: 0.3385 - auc: 0.7879 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Normalizados/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 21s 111ms/step - loss: 0.3221 - auc: 0.8165 - accuracy: 0.8728
[0.322061151266098, 0.8164644241333008, 0.8727818131446838]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Normalizados/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

## Com mais pooling

#### N512-L1-D3

In [ ]:
N = 512
NAME = "N512-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100,
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 47s 189ms/step - loss: 0.4771 - auc: 0.5178 - accuracy: 0.8582 - val_loss: 0.4321 - val_auc: 0.6158 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 47s 191ms/step - loss: 0.4024 - auc: 0.6132 - accuracy: 0.8597 - val_loss: 0.3772 - val_auc: 0.7225 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 46s 186ms/step - loss: 0.3739 - auc: 0.7065 - accuracy: 0.8597 - val_loss: 0.3648 - val_auc: 0.7450 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 47s 190ms/step - loss: 0.3667 - auc: 0.7233 - accuracy: 0.8597 - val_loss: 0.3596 - val_auc: 0.7560 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 46s 186ms/step - loss: 0.3622 - auc: 0.7371 - accuracy: 0.8597 - val_loss: 0.3559 - val_auc: 0.7649 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 46s 186ms/step - loss: 0.3589 - auc: 0.7443 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 34s 183ms/step - loss: 0.3096 - auc: 0.8215 - accuracy: 0.8809
[0.30957847833633423, 0.8215144276618958, 0.8809258341789246]


#### N512-L3-D2

In [ ]:
N = 512
NAME = "N512-L3-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100,
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 65s 205ms/step - loss: 0.4871 - auc: 0.5203 - accuracy: 0.8523 - val_loss: 0.4330 - val_auc: 0.6151 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 42s 171ms/step - loss: 0.3974 - auc: 0.6319 - accuracy: 0.8597 - val_loss: 0.3689 - val_auc: 0.7425 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 42s 169ms/step - loss: 0.3664 - auc: 0.7244 - accuracy: 0.8597 - val_loss: 0.3588 - val_auc: 0.7626 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 43s 175ms/step - loss: 0.3607 - auc: 0.7395 - accuracy: 0.8597 - val_loss: 0.3545 - val_auc: 0.7730 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 42s 170ms/step - loss: 0.3559 - auc: 0.7519 - accuracy: 0.8595 - val_loss: 0.3504 - val_auc: 0.7798 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 43s 175ms/step - loss: 0.3512 - auc: 0.7624 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 22s 113ms/step - loss: 0.3283 - auc: 0.8116 - accuracy: 0.8724
[0.328285276889801, 0.8116336464881897, 0.872353196144104]


#### N1024-L1-D4

In [ ]:
N = 1024
NAME = "N1024-L1-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100,
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 43s 171ms/step - loss: 0.4572 - auc: 0.5590 - accuracy: 0.8582 - val_loss: 0.4197 - val_auc: 0.6124 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 43s 174ms/step - loss: 0.3886 - auc: 0.6547 - accuracy: 0.8597 - val_loss: 0.3646 - val_auc: 0.7510 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 42s 171ms/step - loss: 0.3664 - auc: 0.7261 - accuracy: 0.8597 - val_loss: 0.3564 - val_auc: 0.7649 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 42s 169ms/step - loss: 0.3590 - auc: 0.7457 - accuracy: 0.8597 - val_loss: 0.3516 - val_auc: 0.7753 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 43s 174ms/step - loss: 0.3547 - auc: 0.7558 - accuracy: 0.8597 - val_loss: 0.3480 - val_auc: 0.7813 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 42s 168ms/step - loss: 0.3508 - auc: 0.7634 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 19s 101ms/step - loss: 0.3081 - auc: 0.8273 - accuracy: 0.8780
[0.3080669641494751, 0.8272776007652283, 0.878011167049408]


## Com mais pooling & mais conv

#### N512-L1-D3

In [ ]:
N = 512
NAME = "N512-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                    

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100,
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 49s 193ms/step - loss: 0.6103 - auc: 0.4814 - accuracy: 0.8459 - val_loss: 0.4641 - val_auc: 0.5599 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 48s 192ms/step - loss: 0.4529 - auc: 0.5316 - accuracy: 0.8597 - val_loss: 0.4442 - val_auc: 0.5985 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 56s 229ms/step - loss: 0.4300 - auc: 0.5625 - accuracy: 0.8597 - val_loss: 0.4139 - val_auc: 0.6055 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 48s 193ms/step - loss: 0.3988 - auc: 0.6011 - accuracy: 0.8597 - val_loss: 0.3837 - val_auc: 0.7198 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 47s 191ms/step - loss: 0.3815 - auc: 0.6839 - accuracy: 0.8597 - val_loss: 0.3743 - val_auc: 0.7299 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 47s 192ms/step - loss: 0.3771 - auc: 0.6927 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 19s 102ms/step - loss: 0.3267 - auc: 0.7995 - accuracy: 0.8691
[0.32671087980270386, 0.7995319962501526, 0.8690955638885498]


#### N512-L3-D2

In [ ]:
N = 512
NAME = "N512-L3-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100,
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 45s 174ms/step - loss: 0.4785 - auc: 0.5210 - accuracy: 0.8589 - val_loss: 0.4397 - val_auc: 0.6103 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 41s 167ms/step - loss: 0.4107 - auc: 0.6027 - accuracy: 0.8597 - val_loss: 0.3778 - val_auc: 0.7125 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 43s 173ms/step - loss: 0.3688 - auc: 0.7155 - accuracy: 0.8597 - val_loss: 0.3587 - val_auc: 0.7632 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 41s 168ms/step - loss: 0.3589 - auc: 0.7435 - accuracy: 0.8597 - val_loss: 0.3530 - val_auc: 0.7752 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 42s 168ms/step - loss: 0.3535 - auc: 0.7575 - accuracy: 0.8597 - val_loss: 0.3478 - val_auc: 0.7845 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 41s 167ms/step - loss: 0.3494 - auc: 0.7665 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 21s 108ms/step - loss: 0.3119 - auc: 0.8294 - accuracy: 0.8776
[0.31190958619117737, 0.8293812274932861, 0.8775824904441833]


#### N1024-L1-D4

In [ ]:
N = 1024
NAME = "N1024-L1-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100,
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 47s 186ms/step - loss: 0.4639 - auc: 0.5478 - accuracy: 0.8592 - val_loss: 0.4288 - val_auc: 0.6135 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 44s 178ms/step - loss: 0.3980 - auc: 0.6250 - accuracy: 0.8597 - val_loss: 0.3727 - val_auc: 0.7293 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 44s 178ms/step - loss: 0.3683 - auc: 0.7232 - accuracy: 0.8597 - val_loss: 0.3597 - val_auc: 0.7586 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 44s 179ms/step - loss: 0.3608 - auc: 0.7409 - accuracy: 0.8597 - val_loss: 0.3542 - val_auc: 0.7689 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 44s 178ms/step - loss: 0.3559 - auc: 0.7524 - accuracy: 0.8597 - val_loss: 0.3510 - val_auc: 0.7743 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 45s 181ms/step - loss: 0.3531 - auc: 0.7593 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 20s 106ms/step - loss: 0.3124 - auc: 0.8209 - accuracy: 0.8770
[0.3123529851436615, 0.8208897113800049, 0.8769824504852295]


# Com class_weights

#### N512-L1-D3

In [ ]:
N = 512
NAME = "N512-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:1, 1:5}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 72s 234ms/step - loss: 0.9783 - auc: 0.6881 - accuracy: 0.6707 - val_loss: 0.5615 - val_auc: 0.7580 - val_accuracy: 0.6463
Epoch 2/100
244/244 [==============================] - 57s 229ms/step - loss: 0.9129 - auc: 0.7481 - accuracy: 0.6810 - val_loss: 0.4919 - val_auc: 0.7840 - val_accuracy: 0.7304
Epoch 3/100
244/244 [==============================] - 56s 228ms/step - loss: 0.8716 - auc: 0.7793 - accuracy: 0.7144 - val_loss: 0.5109 - val_auc: 0.7973 - val_accuracy: 0.7087
Epoch 5/100
244/244 [==============================] - 56s 227ms/step - loss: 0.8553 - auc: 0.7900 - accuracy: 0.7204 - val_loss: 0.4949 - val_auc: 0.7994 - val_accuracy: 0.7331
Epoch 6/100
244/244 [==============================] - 57s 229ms/step - loss: 0.8491 - auc: 0.7940 - accuracy: 0.7265 - val_loss: 0.5016 - val_auc: 0.8007 - val_accuracy: 0.7262
Epoch 7/100
244/244 [==============================] - 56s 227ms/step - loss: 0.8333 - auc: 0.8009 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 22s 115ms/step - loss: 0.5218 - auc: 0.8118 - accuracy: 0.7353
[0.5217838883399963, 0.8118057250976562, 0.735276460647583]


#### N512-L1-D3 testando class weights ao contrario

In [ ]:
N = 512
NAME = "N512-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={1:1, 0:5}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 59s 232ms/step - loss: 0.8294 - auc: 0.5736 - accuracy: 0.8590 - val_loss: 0.4688 - val_auc: 0.7484 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 56s 228ms/step - loss: 0.5760 - auc: 0.7304 - accuracy: 0.8597 - val_loss: 0.4365 - val_auc: 0.7748 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 56s 228ms/step - loss: 0.5641 - auc: 0.7541 - accuracy: 0.8597 - val_loss: 0.4560 - val_auc: 0.7865 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 56s 226ms/step - loss: 0.5562 - auc: 0.7693 - accuracy: 0.8597 - val_loss: 0.4430 - val_auc: 0.7908 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 56s 227ms/step - loss: 0.5502 - auc: 0.7785 - accuracy: 0.8597 - val_loss: 0.4501 - val_auc: 0.7951 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 57s 231ms/step - loss: 0.5471 - auc: 0.7822 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 22s 115ms/step - loss: 0.3873 - auc: 0.8096 - accuracy: 0.8624
[0.38727813959121704, 0.8095946907997131, 0.8624089360237122]


#### N1024-L1-D4

In [ ]:
N = 1024
NAME = "N1024-L1-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:1, 1:5},
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 63s 254ms/step - loss: 0.9723 - auc: 0.6941 - accuracy: 0.6750 - val_loss: 0.4922 - val_auc: 0.7691 - val_accuracy: 0.7371
Epoch 2/100
244/244 [==============================] - 62s 251ms/step - loss: 0.9087 - auc: 0.7523 - accuracy: 0.6948 - val_loss: 0.5129 - val_auc: 0.7858 - val_accuracy: 0.7143
Epoch 3/100
244/244 [==============================] - 61s 246ms/step - loss: 0.8835 - auc: 0.7709 - accuracy: 0.7148 - val_loss: 0.5862 - val_auc: 0.7917 - val_accuracy: 0.6464
Epoch 4/100
244/244 [==============================] - 61s 246ms/step - loss: 0.8717 - auc: 0.7773 - accuracy: 0.7185 - val_loss: 0.5764 - val_auc: 0.7969 - val_accuracy: 0.6590
Epoch 5/100
244/244 [==============================] - 61s 248ms/step - loss: 0.8562 - auc: 0.7883 - accuracy: 0.7252 - val_loss: 0.5059 - val_auc: 0.7985 - val_accuracy: 0.7249
Epoch 6/100
244/244 [==============================] - 61s 247ms/step - loss: 0.8417 - auc: 0.7960 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 20s 105ms/step - loss: 0.5143 - auc: 0.8073 - accuracy: 0.7348
[0.5142663717269897, 0.8073198795318604, 0.7347621321678162]


#### N512-L1-D3 com + pooling

In [ ]:
N = 512
NAME = "N512-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:1, 1:5}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 74s 242ms/step - loss: 1.0367 - auc: 0.6163 - accuracy: 0.7224 - val_loss: 0.6132 - val_auc: 0.6938 - val_accuracy: 0.6525
Epoch 2/100
244/244 [==============================] - 47s 190ms/step - loss: 0.9774 - auc: 0.6936 - accuracy: 0.6544 - val_loss: 0.5621 - val_auc: 0.7315 - val_accuracy: 0.6674
Epoch 3/100
244/244 [==============================] - 48s 194ms/step - loss: 0.9441 - auc: 0.7222 - accuracy: 0.6644 - val_loss: 0.5394 - val_auc: 0.7537 - val_accuracy: 0.6716
Epoch 4/100
244/244 [==============================] - 42s 170ms/step - loss: 0.9251 - auc: 0.7380 - accuracy: 0.6732 - val_loss: 0.5352 - val_auc: 0.7675 - val_accuracy: 0.6785
Epoch 5/100
244/244 [==============================] - 44s 179ms/step - loss: 0.9078 - auc: 0.7537 - accuracy: 0.6883 - val_loss: 0.5388 - val_auc: 0.7749 - val_accuracy: 0.6813
Epoch 6/100
244/244 [==============================] - 42s 171ms/step - loss: 0.9025 - auc: 0.7582 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 21s 113ms/step - loss: 0.4767 - auc: 0.8197 - accuracy: 0.7679
[0.47673022747039795, 0.8196592330932617, 0.7678525447845459]


### class_weight definido

In [ ]:
neg = 20047
pos = 3287
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


#### N512-L1-D3

In [ ]:
N = 512
NAME = "N512-L1-D3 cw"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0: w0, 1: w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 78s 257ms/step - loss: 0.6369 - auc: 0.6777 - accuracy: 0.6106 - val_loss: 0.6187 - val_auc: 0.7475 - val_accuracy: 0.6147
Epoch 2/100
244/244 [==============================] - 57s 229ms/step - loss: 0.5953 - auc: 0.7354 - accuracy: 0.6388 - val_loss: 0.5527 - val_auc: 0.7743 - val_accuracy: 0.6647
Epoch 3/100
244/244 [==============================] - 57s 230ms/step - loss: 0.5813 - auc: 0.7554 - accuracy: 0.6554 - val_loss: 0.5078 - val_auc: 0.7809 - val_accuracy: 0.7093
Epoch 4/100
244/244 [==============================] - 57s 231ms/step - loss: 0.5703 - auc: 0.7664 - accuracy: 0.6675 - val_loss: 0.5838 - val_auc: 0.7881 - val_accuracy: 0.6527
Epoch 5/100
244/244 [==============================] - 57s 229ms/step - loss: 0.5644 - auc: 0.7736 - accuracy: 0.6755 - val_loss: 0.5705 - val_auc: 0.7921 - val_accuracy: 0.6666
Epoch 6/100
244/244 [==============================] - 57s 230ms/step - loss: 0.5582 - auc: 0.7808 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 23s 119ms/step - loss: 0.5566 - auc: 0.8036 - accuracy: 0.7084
[0.5566133260726929, 0.8036072850227356, 0.7084440588951111]


#### N512-L3-D2

In [ ]:
N = 512
NAME = "N512-L3-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1},
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 61s 240ms/step - loss: 0.6430 - auc: 0.6824 - accuracy: 0.6002 - val_loss: 0.5727 - val_auc: 0.7334 - val_accuracy: 0.6561
Epoch 2/100
244/244 [==============================] - 58s 236ms/step - loss: 0.5939 - auc: 0.7315 - accuracy: 0.6288 - val_loss: 0.5608 - val_auc: 0.7705 - val_accuracy: 0.6549
Epoch 3/100
244/244 [==============================] - 58s 233ms/step - loss: 0.5743 - auc: 0.7624 - accuracy: 0.6488 - val_loss: 0.4932 - val_auc: 0.7874 - val_accuracy: 0.7336
Epoch 4/100
244/244 [==============================] - 65s 264ms/step - loss: 0.5573 - auc: 0.7831 - accuracy: 0.6774 - val_loss: 0.4931 - val_auc: 0.7971 - val_accuracy: 0.7373
Epoch 5/100
244/244 [==============================] - 58s 234ms/step - loss: 0.5432 - auc: 0.7949 - accuracy: 0.6936 - val_loss: 0.5274 - val_auc: 0.8058 - val_accuracy: 0.7114
Epoch 6/100
244/244 [==============================] - 57s 232ms/step - loss: 0.5257 - auc: 0.8129 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 19s 102ms/step - loss: 0.5662 - auc: 0.8153 - accuracy: 0.7159
[0.5661687850952148, 0.8153330683708191, 0.7159022688865662]


#### N1024-L1-D4

In [ ]:
N = 1024
NAME = "N1024-L1-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1},
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 62s 248ms/step - loss: 0.6242 - auc: 0.6973 - accuracy: 0.6315 - val_loss: 0.5551 - val_auc: 0.7667 - val_accuracy: 0.6664
Epoch 2/100
244/244 [==============================] - 61s 249ms/step - loss: 0.5770 - auc: 0.7613 - accuracy: 0.6558 - val_loss: 0.5800 - val_auc: 0.7910 - val_accuracy: 0.6549
Epoch 3/100
244/244 [==============================] - 61s 246ms/step - loss: 0.5588 - auc: 0.7809 - accuracy: 0.6797 - val_loss: 0.6240 - val_auc: 0.7992 - val_accuracy: 0.6295
Epoch 4/100
244/244 [==============================] - 61s 247ms/step - loss: 0.5462 - auc: 0.7921 - accuracy: 0.6925 - val_loss: 0.6103 - val_auc: 0.8045 - val_accuracy: 0.6459
Epoch 5/100
244/244 [==============================] - 61s 246ms/step - loss: 0.5361 - auc: 0.8023 - accuracy: 0.7032 - val_loss: 0.5584 - val_auc: 0.8081 - val_accuracy: 0.6850
Epoch 6/100
244/244 [==============================] - 69s 281ms/step - loss: 0.5253 - auc: 0.8105 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 20s 104ms/step - loss: 0.5538 - auc: 0.8154 - accuracy: 0.7150
[0.5537827014923096, 0.8153848648071289, 0.7149592638015747]


## Com mais pooling

#### N512-L1-D3

In [ ]:
N = 512
NAME = "N512-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 43s 169ms/step - loss: 0.6664 - auc: 0.6493 - accuracy: 0.6083 - val_loss: 0.6655 - val_auc: 0.6890 - val_accuracy: 0.6158
Epoch 2/100
244/244 [==============================] - 42s 169ms/step - loss: 0.6308 - auc: 0.6859 - accuracy: 0.6160 - val_loss: 0.6410 - val_auc: 0.7179 - val_accuracy: 0.6160
Epoch 3/100
244/244 [==============================] - 41s 167ms/step - loss: 0.6114 - auc: 0.7097 - accuracy: 0.6224 - val_loss: 0.5963 - val_auc: 0.7423 - val_accuracy: 0.6345
Epoch 4/100
244/244 [==============================] - 42s 168ms/step - loss: 0.5981 - auc: 0.7300 - accuracy: 0.6293 - val_loss: 0.5571 - val_auc: 0.7562 - val_accuracy: 0.6563
Epoch 5/100
244/244 [==============================] - 42s 168ms/step - loss: 0.5884 - auc: 0.7458 - accuracy: 0.6369 - val_loss: 0.5948 - val_auc: 0.7674 - val_accuracy: 0.6314
Epoch 6/100
244/244 [==============================] - 42s 168ms/step - loss: 0.5820 - auc: 0.7535 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 19s 102ms/step - loss: 0.5008 - auc: 0.8223 - accuracy: 0.7597
[0.5007873177528381, 0.8222819566726685, 0.7597085237503052]


#### N512-L3-D2

In [ ]:
N = 512
NAME = "N512-L3-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1},
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 63s 197ms/step - loss: 0.6692 - auc: 0.6463 - accuracy: 0.5237 - val_loss: 0.6827 - val_auc: 0.6926 - val_accuracy: 0.5989
Epoch 2/100
244/244 [==============================] - 47s 191ms/step - loss: 0.6337 - auc: 0.6834 - accuracy: 0.6066 - val_loss: 0.6183 - val_auc: 0.7260 - val_accuracy: 0.6319
Epoch 3/100
244/244 [==============================] - 46s 188ms/step - loss: 0.6127 - auc: 0.7146 - accuracy: 0.6210 - val_loss: 0.5769 - val_auc: 0.7522 - val_accuracy: 0.6557
Epoch 4/100
244/244 [==============================] - 47s 191ms/step - loss: 0.5961 - auc: 0.7381 - accuracy: 0.6351 - val_loss: 0.5978 - val_auc: 0.7684 - val_accuracy: 0.6273
Epoch 5/100
244/244 [==============================] - 46s 188ms/step - loss: 0.5865 - auc: 0.7509 - accuracy: 0.6448 - val_loss: 0.5939 - val_auc: 0.7773 - val_accuracy: 0.6432
Epoch 6/100
244/244 [==============================] - 47s 188ms/step - loss: 0.5803 - auc: 0.7587 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 21s 113ms/step - loss: 0.5362 - auc: 0.8151 - accuracy: 0.7385
[0.5362021923065186, 0.8151167631149292, 0.7385340929031372]


#### N1024-L1-D4

In [ ]:
N = 1024
NAME = "N1024-L1-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1},
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 53s 211ms/step - loss: 0.6623 - auc: 0.6585 - accuracy: 0.5984 - val_loss: 0.6449 - val_auc: 0.6960 - val_accuracy: 0.6287
Epoch 2/100
244/244 [==============================] - 48s 193ms/step - loss: 0.6226 - auc: 0.6967 - accuracy: 0.6211 - val_loss: 0.6170 - val_auc: 0.7311 - val_accuracy: 0.6314
Epoch 3/100
244/244 [==============================] - 49s 197ms/step - loss: 0.6033 - auc: 0.7225 - accuracy: 0.6298 - val_loss: 0.6374 - val_auc: 0.7544 - val_accuracy: 0.6153
Epoch 4/100
244/244 [==============================] - 42s 171ms/step - loss: 0.5908 - auc: 0.7407 - accuracy: 0.6360 - val_loss: 0.5844 - val_auc: 0.7672 - val_accuracy: 0.6446
Epoch 5/100
244/244 [==============================] - 42s 169ms/step - loss: 0.5838 - auc: 0.7519 - accuracy: 0.6429 - val_loss: 0.5581 - val_auc: 0.7739 - val_accuracy: 0.6665
Epoch 6/100
244/244 [==============================] - 42s 170ms/step - loss: 0.5783 - auc: 0.7603 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 19s 100ms/step - loss: 0.5365 - auc: 0.8113 - accuracy: 0.7271
[0.5365164279937744, 0.8113262057304382, 0.7271324396133423]


## Com mais pooling & mais conv

#### N512-L1-D3

In [ ]:
N = 512
NAME = "N512-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                    

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 49s 193ms/step - loss: 0.6912 - auc: 0.5532 - accuracy: 0.3739 - val_loss: 0.6986 - val_auc: 0.6568 - val_accuracy: 0.3024
Epoch 2/100
244/244 [==============================] - 47s 189ms/step - loss: 0.6878 - auc: 0.6311 - accuracy: 0.4282 - val_loss: 0.6994 - val_auc: 0.6910 - val_accuracy: 0.5397
Epoch 3/100
244/244 [==============================] - 46s 188ms/step - loss: 0.6745 - auc: 0.6570 - accuracy: 0.5601 - val_loss: 0.6798 - val_auc: 0.6817 - val_accuracy: 0.5981
Epoch 4/100
244/244 [==============================] - 46s 187ms/step - loss: 0.6475 - auc: 0.6652 - accuracy: 0.5978 - val_loss: 0.6625 - val_auc: 0.6908 - val_accuracy: 0.6005
Epoch 5/100
244/244 [==============================] - 46s 187ms/step - loss: 0.6317 - auc: 0.6834 - accuracy: 0.6022 - val_loss: 0.6482 - val_auc: 0.7075 - val_accuracy: 0.6022
Epoch 6/100
244/244 [==============================] - 48s 193ms/step - loss: 0.6220 - auc: 0.6978 - accuracy:

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

#### N512-L3-D2

In [ ]:
N = 512
NAME = "N512-L3-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1},
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)

#### N1024-L1-D4

In [ ]:
N = 1024
NAME = "N1024-L1-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(normalized_data, validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1},
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

In [ ]:
results = model.evaluate(normalized_test_data)
print(results)